In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("../dataset")

In [3]:
x_columns = data.columns.drop('label')
x = data[x_columns]
x = x.values
y = data['label']

le_proto = LabelEncoder()
x[:, 0] = le_proto.fit_transform(x[:, 0])
# le_label = LabelEncoder()
# y = le_label.fit_transform(y)
sc = StandardScaler()
x=sc.fit_transform(x)


In [4]:
clf = svm.SVC()

clf.fit(x, y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [5]:
datatest = pd.read_csv("../datatest")

x_columns = datatest.columns.drop('label')
x = datatest[x_columns]
x = x.values
y = datatest['label']

x[:, 0] = le_proto.transform(x[:, 0])
#joblib.dump(le_proto, '../preproc/le_proto2.joblib',protocol=1) #save label encoder weights to file
# y = le_label.fit_transform(y)
#joblib.dump(le_label, '../preproc/le_label2.joblib',protocol=1) #save label encoder weights to file
x=sc.transform(x)
y_pred = clf.predict(x)
print("Accuracy:",metrics.accuracy_score(y, y_pred))

Accuracy: 0.9794681925277684


In [6]:
stats = {}
labels = ['normal','tcpsynflood','icmpflood','udpflood','ipsweep','portscan','pingofdeath']
for i in labels:
    stats[i]={}
    for j in labels:
        stats[i][j] = 0
for i in range(len(y)):
    stats[y[i]][y_pred[i]] += 1
stats

{'normal': {'normal': 1157,
  'tcpsynflood': 0,
  'icmpflood': 72,
  'udpflood': 12,
  'ipsweep': 0,
  'portscan': 1,
  'pingofdeath': 0},
 'tcpsynflood': {'normal': 6,
  'tcpsynflood': 1039,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 0},
 'icmpflood': {'normal': 0,
  'tcpsynflood': 0,
  'icmpflood': 793,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 0},
 'udpflood': {'normal': 0,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 825,
  'ipsweep': 0,
  'portscan': 1,
  'pingofdeath': 0},
 'ipsweep': {'normal': 1,
  'tcpsynflood': 0,
  'icmpflood': 3,
  'udpflood': 0,
  'ipsweep': 646,
  'portscan': 0,
  'pingofdeath': 0},
 'portscan': {'normal': 16,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 3,
  'ipsweep': 0,
  'portscan': 899,
  'pingofdeath': 0},
 'pingofdeath': {'normal': 0,
  'tcpsynflood': 0,
  'icmpflood': 7,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 461}}

In [7]:
tpr={}
fpr={}
precision={}
f1score={}
for i in labels:
    tp = stats[i][i]
    fp = 0
    tn = 0
    fn = 0
    for j in labels:
        if i != j:
            fn += stats[i][j]
            fp += stats[j][i]
            for k in stats[j].keys():
                if k != i:
                    tn += stats[j][k]
    tpr[i] = tp / (tp + fn) * 100
    fpr[i] = fp / (fp + tn) * 100
    precision[i] = tp / (tp + fp) * 100
    f1score[i] = 2/(1/tpr[i] + 1/precision[i])
print('TPR: \n ' + str(tpr))
print('FRP:\n ' + str(fpr))
# print('Precision:\n ' + str(precision))
print('F1-score: \n' + str(f1score))


print("Accuracy:",metrics.accuracy_score(y, y_pred))

TPR: 
 {'normal': 93.15619967793882, 'tcpsynflood': 99.42583732057416, 'icmpflood': 100.0, 'udpflood': 99.87893462469734, 'ipsweep': 99.38461538461539, 'portscan': 97.93028322440087, 'pingofdeath': 98.50427350427351}
FRP:
 {'normal': 0.48936170212765956, 'tcpsynflood': 0.0, 'icmpflood': 1.5925422412118857, 'udpflood': 0.29319781078967944, 'ipsweep': 0.0, 'portscan': 0.039808917197452234, 'pingofdeath': 0.0}
F1-score: 
{'normal': 95.54087530966143, 'tcpsynflood': 99.71209213051824, 'icmpflood': 95.08393285371702, 'udpflood': 99.03961584633852, 'ipsweep': 99.69135802469135, 'portscan': 98.84551951621769, 'pingofdeath': 99.24650161463941}
Accuracy: 0.9794681925277684
